In [1]:
import numpy as np
import datetime 
import matplotlib.pyplot as plt
import os
import json

OUT_DAY = 'arr_day.npy' #the output files containing integrated power and metadata for each sweep-spectrum on record
OUT_NIGHT = 'arr_night.npy'
#DATA_PATH = #
SUNSET_TIMETABLE = np.genfromtxt('/lustre/aoc/projects/hera/ajosaiti/SDR_RFI_monitoring/HERA_daily_RFI/HERA_sunrise_sunset_annual.csv', dtype=str,delimiter=',') # Col 0: Month-day (mmdd), Col 1: Sunrise (hhmm), Col 2: Sunset (hhmm)
STR_DAY = os.environ['which_day']
DEBUG = True

def file_flush():
	if DEBUG: print('flushing temporary and output files...')
	open(OUT_DAY, 'w').close()
	open(OUT_NIGHT, 'w').close()
	if DEBUG: print('done.')

def delta_hours_minutes(td):
    return td.seconds//3600, (td.seconds//60)%60

def recursive_key_search(dat, key): #https://stackoverflow.com/questions/9807634/find-all-occurrences-of-a-key-in-nested-python-dictionaries-and-lists
#sum_dbm_recursive_key_search(dat, key): #https://stackoverflow.com/questions/9807634/find-all-occurrences-of-a-key-in-nested-python-dictionaries-and-lists
    if key in dat:
        yield dat[key]
    for k in dat:
        if isinstance(dat[k], list):
            for i in dat[k]:
                for j in recursive_key_search(i):
                    yield j

def time_in_rids_fmt(datetime_time): # convert datetime.datetime.now() time into the RIDZ-requested format
	str_iso = datetime_time.isoformat(' ')
	str_time_rids = str( str_iso[0:4] + str_iso[5:7] + str_iso[8:10] + '-' + str_iso[11:13]+str_iso[14:16]+str_iso[17:19])
	return str_time_rids

def sum_array_of_dbm(arr):
	val_sum = 10.*np.log10(np.sum(np.power(10,np.array(arr)/10.)))
	return val_sum


  

def day_night_initial_calculation(arr_day,arr_night):
    ## connect to librarian
    #cl = LibrarianClient(connection_name)

    ## search for unprocessed sessions
    #files = cl.search_files(search)['results']

    #if not len(sessions):
    #    return # Nothing to do.

    #plot_script = os.path.join(plots_dir, 'run_notebook.sh')

    ## check these sessid aren't in the processed_sessid.txt file
    plots_dir = os.path.dirname(os.environ['DATA_PATH'])
    processed_fileid = np.loadtxt(os.path.join(plots_dir, 'processed_fileid.txt'), dtype=np.int)
    if DEBUG: 
        print('NOTE: Since in DEBUG mode, ignoring which files have already been processed. Will reprocess if instructed.')
        
    # filter out sessions already processed
    unprocessed_files = []
    pathname = []
    for diritem in os.listdir(os.environ['staging_dir']):
        if DEBUG: print('in ipynb, staging_dir is: '+str(os.environ['staging_dir']))
        if os.path.isdir(diritem) or str(diritem).isdigit():
            pathname = str(str(os.environ['staging_dir'])+'/'+str(diritem)+'/')
            for dir_file in os.listdir(pathname):
                name=str(dir_file)#file["name"])
                if DEBUG: print('in diritem / dif_file, pathname: '+str(pathname)+', name: '+str(name))
                fileid = int(name[int(name.find('.')+1):name.find('-')])
                if (fileid not in processed_fileid) or DEBUG:
                    unprocessed_files.append(name)
        else:
            pathname = os.environ['staging_dir']
            if DEBUG: 
                name=str(diritem)#file["name"])
                print(str(name)+' is thought not to be a directory, or name only contained digits (so a strong suspicion of being directory).')
            print('when diritem itself is a file, name: '+str(name))
            fileid = int(name[int(name.find('.')+1):name.find('-')])
            if (fileid not in processed_fileid) or DEBUG:
                unprocessed_files.append(name)

    #cwd = os.getcwd()
    if DEBUG: 
        print('List of unprocessed files (or all files if in DEBUG): '+str(unprocessed_files))
        print('ready for analysis with files in the following pathname: '+str(pathname))
    for filename in os.listdir(pathname):
        if DEBUG: print('filename: '+str(filename))
        if str(filename) in unprocessed_files:
            if DEBUG: print('filename is in unprocessed_files.')
            if filename.endswith('.ridz') and (STR_DAY in filename): 
                fname_uzip = str(filename.replace('.ridz','.rids'))
                os.system(str('zipr.py ' +str(str(pathname)+str(filename))))
                dat = json.loads(open(str(str(pathname)+str(fname_uzip))).read())
                for dset in np.array(dat['feature_sets'].keys()):
                    t_spectra = datetime.datetime.strptime(str(dset[5:20]), '%Y%m%d-%H%M%S' )
                    int_pwr =  sum_array_of_dbm(dat['feature_sets'][str(dset)]['val'])# scalar, the integrated power from one sweep spectrogram measurement
                    if DEBUG: 
                        print('int_pwr: '+str(int_pwr))
                        print('t_spectra: '+str(t_spectra))
                        print('sunrise: '+str(sunrise)+', sunset: '+str(sunset))
                    if bool( (t_spectra >=sunrise) and (t_spectra<=sunset) ): arr_day.append(np.array([t_spectra,int_pwr]))
                    else: arr_night.append(np.array([t_spectra,int_pwr]))
                dat.clear()
                os.system(str('zipr.py ' +str(fname_uzip))) # re-zip the file
        else: print('str(filename) was not in unprocessed_files array. Skipping.')        
    if DEBUG:
        print('arr_day: '+str(arr_day))
        print('arr_night: '+str(arr_night))

    return arr_day, arr_night

#For the day in question, figure out when sunset and sunrise were. 
col_date,col_key = np.where(SUNSET_TIMETABLE == STR_DAY[4:]) #find mmdd, but it could also be a time
sunrise = datetime.datetime.strptime( str(str(STR_DAY[:4])+str(STR_DAY[4:])+'-'+str(SUNSET_TIMETABLE[int(col_date[col_key==0])][1])) ,'%Y%m%d-%H%M')
sunset = datetime.datetime.strptime( str(str(STR_DAY[:4])+str(STR_DAY[4:])+'-'+str(SUNSET_TIMETABLE[int(col_date[col_key==0])][2])) ,'%Y%m%d-%H%M')
delta_hour, delta_minute = delta_hours_minutes((sunset-sunrise))
#Arrays containing data from each sweep measurement, sorted by whether the data occured in the daytime or nighttime.
arr_day=[] # [datetime.datetime object], [float(integrated power)]
arr_night=[]


file_flush() # Make sure output files aren't reused with old data in them.
arr_day, arr_night = day_night_initial_calculation(arr_day,arr_night)

#Turn the array of arrays into a 2D array
arr_day=np.array(arr_day)
arr_night = np.array(arr_night)

if DEBUG: 
    print('arr_day shape: ' +str(np.shape(arr_day)))
    print('arr_night shape: ' +str(np.shape(arr_night)))

np.save(OUT_DAY,arr_day)
np.save(OUT_NIGHT,arr_night)


if DEBUG: 
    print('arr_day: '+str(arr_day))
    print('arr_day[0,0]: '+str(arr_day[0,0]))
    print('arr_day[:,0]: '+str(arr_day[:,0]))
    
arr_full_day = []
if (np.size(arr_day) !=0) and (np.size(arr_night) !=0): arr_full_day = np.concatenate((np.array(arr_day), np.array(arr_night)),axis=0)
elif (np.size(arr_day) ==0): arr_full_day = arr_night
else: arr_full_day = arr_day
    
plt.figure(1, figsize=(20,10))
plt.subplot(211)
plt.plot(arr_day[:,0], arr_day[:,1],'bo')
plt.plot(arr_night[:,0], arr_night[:,1],'go')
plt.subplot(211).set_title(str('Daytime ('+str(delta_hour)+' Hours, '+str(delta_minute)+' Minutes) Integrated Power: '+str(  np.round(sum_array_of_dbm(arr_day[:,1]),decimals=1) )+ 'dBm, Nighttime Integrated Power: '+str(  np.round(sum_array_of_dbm(arr_night[:,1]),decimals=1)  )+' dBm'))
plt.subplot(212)
plt.plot(arr_full_day[:,0], arr_full_day[:,1],'ro')
plt.subplot(212).set_title(str('Full 24 Hours, Average Integrated Power: ' + str(  np.round(sum_array_of_dbm(arr_full_day[:,1]),decimals=1)  )+' dBm'))

 
plt.show()

flushing temporary and output files...
done.
NOTE: Since in DEBUG mode, ignoring which files have already been processed. Will reprocess if instructed.
in ipynb, staging_dir is: /lustre/aoc/projects/hera/ajosaiti/SDR_RFI_monitoring_staging/20180920
in diritem / dif_file, pathname: /lustre/aoc/projects/hera/ajosaiti/SDR_RFI_monitoring_staging/20180920/2458706/, name: SDR_SpectrumPeak.20180920-172630.n6.None.ridz
List of unprocessed files (or all files if in DEBUG): ['SDR_SpectrumPeak.20180920-172630.n6.None.ridz']
ready for analysis with files in the following pathname: /lustre/aoc/projects/hera/ajosaiti/SDR_RFI_monitoring_staging/20180920/2458706/
filename: SDR_SpectrumPeak.20180920-172630.n6.None.ridz
filename is in unprocessed_files.
int_pwr: -64.60833922070962
t_spectra: 2018-09-20 17:29:00
sunrise: 2018-09-20 05:27:00, sunset: 2018-09-20 17:29:00
int_pwr: -64.61281471479705
t_spectra: 2018-09-20 17:28:00
sunrise: 2018-09-20 05:27:00, sunset: 2018-09-20 17:29:00
int_pwr: -64.5460448

IndexError: too many indices for array